In [1]:
import numpy as np
import pandas as pd

In [2]:
full_dataset = pd.read_csv('full_dataset_1.1.csv', encoding='latin', low_memory=False)
print("Id_x Column Facts:")
print("Unique Values:", full_dataset['Id_x'].nunique())
print("Null Values:", full_dataset['Id_x'].isna().sum())
full_dataset.shape

Id_x Column Facts:
Unique Values: 30540
Null Values: 0


(30754, 425)

First, we see that most of data is clean, where Mailing state is encoded by 2-letter common abbreviation of state. In addition there is a bunch of garbage data, which typically represent no more than 10 records (most are just 1), including those located outside of US. It is prudent to limit research to only those records that can be easily attributed to one of 50 states in USA and ignore the rest of data. By doing this we lose about 500 records still leaving hefty 29K records to analyze.

In [3]:
full_dataset.MailingState.value_counts()

TX                          3885
CA                          3170
VA                          2547
NC                          2532
CO                          2449
GA                          2229
FL                          1886
WA                          1344
NY                           851
MD                           609
TN                           588
SC                           496
AZ                           493
ID                           484
PA                           462
AL                           453
HI                           442
IL                           422
NJ                           386
KS                           340
OH                           335
MO                           310
KY                           298
OK                           262
LA                           250
MI                           250
NV                           231
IN                           230
MA                           172
MS                           142
          

List of states abbreviation was taken from Wikipedia to create a mask (list of 2-letters abbreviation) that we can use to filter our data. A snippet of such list is shown below.

In [4]:
states_dict = pd.read_csv('States_dict.csv', names=['State_abr', 'State_name'])
states_dict.head()

,State_abr,State_name
0,AL,Alabama
1,AK,Alaska
2,AZ,Arizona
3,AR,Arkansas
4,CA,California


Creating a copy of dataframe to perfmorm all transformations.

In [5]:
clean_data = full_dataset.copy()

Converting all records in Mailing_state column to uppercase to aovid misses from direct string comparison.
"Filtering" dataframe to include only those records that are present in true list of 50 states abbreviations.

In [6]:
clean_data['MailingState'] = clean_data['MailingState'].str.upper()
clean_data = clean_data[clean_data['MailingState'].isin(states_dict['State_abr'])]

After filtering we lost about 700 records

In [7]:
len(full_dataset) - len(clean_data)

661

"Highest education attainment" column also mostly has clean data with 6 categories that dominate: High School, Associate, 2 yr Degree, 4 yr Degree, Masters and Doctorate. Such categories can well be mapped to commonly used taxonomy. In addition, there are records (of 9 occurences and less) that, though contain some information, couldn't be easily mapped to common categories. Dropping those "hard to deal with" records looks like a resonable compromise.

In [8]:
clean_data['Highest_Level_of_Education_Completed__c'].value_counts()

High School/GED                                                                                                                                                                                                                                  9789
4 Year Degree (BA, BS, etc.)                                                                                                                                                                                                                     8696
Post-Graduate Degree (MA, MS, JD, etc.)                                                                                                                                                                                                          4716
2 Year Degree (AA, AS, etc.)                                                                                                                                                                                                                     4528
Doctorate (PhD, 

Grouped list of occurences is stored in "temp_ind" variable. It is further reduced to a list where no. of occurences is over 10. It looks arbitrary but eventually it leaves us with the desired 6-categories that are commonly used.

In [9]:
temp_ind = clean_data['Highest_Level_of_Education_Completed__c'].value_counts()
temp_ind = temp_ind.loc[temp_ind > 10]

Finally cleaning the master dataframe to include only 6 categories for educational attainment and non missing values. By doing so we lose about 2K records.

In [11]:
clean_data = clean_data[clean_data['Highest_Level_of_Education_Completed__c'].isin(temp_ind.index)]
len(full_dataset) - len(clean_data)

2798

After getting rid of all NaN, very low occurence or corrupted data a clean data set still contains almost 28K records.

In [12]:
len(clean_data)

27956

Saving end result to a file.

In [13]:
clean_data.to_csv("full_dataset_1.1.MK.csv", index=False)